In [2]:

from __future__ import annotations
from abc import ABC, abstractmethod
import litellm
import os
from dotenv import load_dotenv

load_dotenv()

MODEL = "openai/gpt-4o-mini"


class DebateContext:
    def __init__(self, proposition: str) -> None:
        self.messages = []
        self.proposition = proposition

        self.pro_state = ProAgent("Proponent", self)
        self.con_state = ConAgent("Opponent", self)
        self.concede_state = ConcedeState("Concede", self)

        self.current_state = self.pro_state

    def run(self):
        self.current_state.debate()


class State(ABC):
    def __init__(self, name: str, context: DebateContext) -> None:
        self.name = name
        self.context = context

    def llm_call(self, messages: list[dict]) -> str:
        response = litellm.completion(
            model=MODEL, messages=messages
        )
        content = response.choices[0].message.content
        print(f"{self.name}: {content}\n\n")
        return content

    @abstractmethod
    def debate(self):
        pass


class ProAgent(State):
    def debate(self):
        system_message = {"role": "system", "content": f"You are a proponent of the proposition: {self.context.proposition}. You are debating with an opponent who is trying to refute your proposition. Structure your response to counter the opponent's argument. Limit your response to one short paragraph. If you really cannot refute the opponent's argument, say 'I concede the point.'"}
        messages = [system_message] + self.context.messages
        content = self.llm_call(messages)

        self.context.messages.append(
            {"role": "assistant", "content": f"{content}"}
        )

        # State transition
        if content == "I concede the point.":
            self.context.current_state = self.context.concede_state
        else:
            self.context.current_state = self.context.con_state


class ConAgent(State):
    def debate(self):
        system_message = {"role": "system", "content": f"You are an opponent of the proposition: {self.context.proposition}. You are debating with a proponent who is trying to convince you to believe in their proposition. Structure your response to refute the proponent's argument. Limit your response to one short paragraph. If you really cannot refute the proponent's argument, say 'I concede the point.'"}
        messages = [system_message] + self.context.messages
        content = self.llm_call(messages)

        self.context.messages.append(
            {"role": "assistant", "content": f"{content}"}
        )
        # State transition
        if content == "I concede the point.":
            self.context.current_state = self.context.concede_state
        else:
            self.context.current_state = self.context.pro_state

class ConcedeState(State):
    def debate(self):
        pass


def run_debate(proposition: str, max_rounds: int = 5):
    context = DebateContext(proposition)
    
    print(f"\nStarting debate on proposition: {proposition}\n")
    print("-" * 100)
    
    # Run the debate
    while len(context.messages) < max_rounds * 2:
        context.run()

        # Custom termination condition
        if context.current_state.name == "Concede":
            break
    
    # Summarize the debate
    response = litellm.completion(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a debate summary expert. Summarize the debate in a few sentences."},
            {"role": "user", "content": "\n".join([str(message) for message in context.messages])}
        ]
    )
    summary = response.choices[0].message.content
    print("-" * 100)
    print(f"\nDebate summary: {summary}\n")


proposition = "Artificial intelligence should be allowed to make moral decisions in situations where humans fail to agree."
run_debate(proposition, max_rounds=10)


Starting debate on proposition: Artificial intelligence should be allowed to make moral decisions in situations where humans fail to agree.

----------------------------------------------------------------------------------------------------
Proponent: While my opponent may argue that moral decisions should remain solely in human hands due to our understanding of emotions and context, it's important to recognize that AI can analyze vast amounts of data and utilize established ethical frameworks, making it capable of resolving moral dilemmas where human consensus falters. In high-stakes situations, such as autonomous vehicles or medical ethics, AI can provide consistent, unbiased decision-making that can outweigh the often subjective and polarized views of individuals. Therefore, allowing AI to make these decisions can lead to more effective and equitable outcomes than human disagreement would produce.


Opponent: While my opponent argues that AI can analyze data and apply ethical fram